In [1]:
from numba import cuda
import matplotlib.pyplot as plt
import numpy as np
import time
import math
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
im = plt.imread("/content/drive/MyDrive/Colab Notebooks/image6.jpeg")

In [5]:
shape = np.shape(im)

In [6]:
filter = np.array(((0,0,1,2,1,0,0),
(0,3,13,22,13,3,0),
(1,13,59,97,59,13,1),
(2,22,97,159,97,22,2),
(1,13,59,97,59,13,1),
(0,3,13,22,13,3,0),
(0,0,1,2,1,0,0)))

In [7]:
total = sum(sum(filter))
total

1003

In [8]:
filter2 = filter/total
filter2

array([[0.        , 0.        , 0.00099701, 0.00199402, 0.00099701,
        0.        , 0.        ],
       [0.        , 0.00299103, 0.01296112, 0.0219342 , 0.01296112,
        0.00299103, 0.        ],
       [0.00099701, 0.01296112, 0.05882353, 0.09670987, 0.05882353,
        0.01296112, 0.00099701],
       [0.00199402, 0.0219342 , 0.09670987, 0.15852443, 0.09670987,
        0.0219342 , 0.00199402],
       [0.00099701, 0.01296112, 0.05882353, 0.09670987, 0.05882353,
        0.01296112, 0.00099701],
       [0.        , 0.00299103, 0.01296112, 0.0219342 , 0.01296112,
        0.00299103, 0.        ],
       [0.        , 0.        , 0.00099701, 0.00199402, 0.00099701,
        0.        , 0.        ]])

In [9]:
im2 = im.copy()

In [10]:
#first version CPU
"""
for x in range(3,shape[0]-3):
  for y in range(3,shape[1]-3):
    r = 0
    g = 0
    b = 0

    for ni, i in enumerate(range(-3,3)):
        for nj, j in enumerate(range(-3,3)):
          r+=(im[x+i,y+j,0]*filter2[ni,nj])
          g+=(im[x+i,y+j,1]*filter2[ni,nj])
          b+=(im[x+i,y+j,2]*filter2[ni,nj])

    im2[x,y,0] = r
    im2[x,y,1] = g
    im2[x,y,2] = b
"""

'\nfor x in range(3,shape[0]-3):\n  for y in range(3,shape[1]-3):\n    r = 0\n    g = 0\n    b = 0\n\n    for ni, i in enumerate(range(-3,3)):\n        for nj, j in enumerate(range(-3,3)):\n          r+=(im[x+i,y+j,0]*filter2[ni,nj])\n          g+=(im[x+i,y+j,1]*filter2[ni,nj])\n          b+=(im[x+i,y+j,2]*filter2[ni,nj])\n\n    im2[x,y,0] = r\n    im2[x,y,1] = g\n    im2[x,y,2] = b\n\n'

In [11]:
"""
imgpu = Image.fromarray(im2)
imgpu.save("/content/drive/MyDrive/Colab Notebooks/image_blur_CPU1.jpeg")
"""

'\nimgpu = Image.fromarray(im2)\nimgpu.save("/content/drive/MyDrive/Colab Notebooks/image_blur_CPU1.jpeg")\n'

In [23]:
im3 = im.copy()

In [24]:
#Second version CPU
def blurfunc(image):
  imagecopy = image
  for x in range(3,shape[0]-3):
    for y in range(3,shape[1]-3):
      rgb=[0,0,0]
      for ni, i in enumerate(range(-3,3)):
        for nj, j in enumerate(range(-3,3)):
          rgb+=(image[x+i,y+j]*filter2[ni,nj])
          
      imagecopy[x,y] = rgb

  return imagecopy
  

In [25]:
for i in range(10):
  im3 = blurfunc(im3)

imgpu = Image.fromarray(im3)
imgpu.save("/content/drive/MyDrive/Colab Notebooks/image_blur_CPU3.jpeg")

In [ ]:
#GPU1

@cuda.jit
def blurgpu1(kernel, src, dst):
  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y

  

  blur = np.uint8(src[tidx, tidy]*)
  dst[tidx, tidy] = g